In [1]:
from fastai.collab import *
from fastai.tabular.all import *
path = untar_data(URLs.ML_100k)
path.ls()

(#23) [Path('C:/Users/liude/.fastai/data/ml-100k/allbut.pl'),Path('C:/Users/liude/.fastai/data/ml-100k/mku.sh'),Path('C:/Users/liude/.fastai/data/ml-100k/README'),Path('C:/Users/liude/.fastai/data/ml-100k/u.data'),Path('C:/Users/liude/.fastai/data/ml-100k/u.genre'),Path('C:/Users/liude/.fastai/data/ml-100k/u.info'),Path('C:/Users/liude/.fastai/data/ml-100k/u.item'),Path('C:/Users/liude/.fastai/data/ml-100k/u.occupation'),Path('C:/Users/liude/.fastai/data/ml-100k/u.user'),Path('C:/Users/liude/.fastai/data/ml-100k/u1.base')...]

In [2]:
ratings = pd.read_csv(path/'u.data', delimiter='\t', header=None,
                      names = ['user', 'movie', 'rating', 'timestamp'])
ratings.head()

,user,movie,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [3]:
last_skywalker = np.array([.98, 0.9, -0.9])
user1 = np.array([0.9, 0.8, -0.6])
(user1 * last_skywalker).sum()

2.1420000000000003

In [4]:
casablanca = np.array([-0.99, -0.3, 0.8])
(user1 * casablanca).sum()

-1.611

In [5]:
movies = pd.read_csv(path/'u.item', delimiter='|', encoding='latin-1', 
                     usecols=(0,1), names=('movie', 'title'), header=None)
movies.head()

,movie,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [6]:
ratings = ratings.merge(movies)
ratings.head()

,user,movie,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,186,302,3,891717742,L.A. Confidential (1997)
2,22,377,1,878887116,Heavyweights (1994)
3,244,51,2,880606923,Legends of the Fall (1994)
4,166,346,1,886397596,Jackie Brown (1997)


In [7]:
dls = CollabDataLoaders.from_df(ratings, item_name='title', bs=64)
dls.show_batch()

,user,title,rating
0,346,Showgirls (1995),1
1,648,Terminator 2: Judgment Day (1991),5
2,406,Bio-Dome (1996),2
3,844,Eat Drink Man Woman (1994),4
4,630,Beavis and Butt-head Do America (1996),3
5,934,Strictly Ballroom (1992),3
6,527,"Third Man, The (1949)",4
7,13,Blade Runner (1982),4
8,433,"Usual Suspects, The (1995)",5
9,409,"Godfather: Part II, The (1974)",3


In [8]:
n_users = len(dls.classes['user'])
n_movies = len(dls.classes['title'])
n_factors = 5
user_factors = torch.randn(n_users, n_factors)
movie_factors = torch.randn(n_movies, n_factors)
n_users, n_movies

(944, 1665)

In [7]:
one_hot_3 = one_hot(3, n_users).float()
user_factors.t() @ one_hot_3

tensor([ 1.0127,  1.4288, -1.6019, -1.6063, -0.4847])

In [8]:
user_factors[3]

tensor([ 1.0127,  1.4288, -1.6019, -1.6063, -0.4847])

In [13]:
class DotProduct(Module):
    def __init__(self, n_users, n_movies, n_factors):
        super().__init__()
        self.user_factors = Embedding(n_users, n_factors)
        self.movie_factors = Embedding(n_movies, n_factors)
        
    def forward(self, x):
        users = self.user_factors(x[:, 0])
        movies = self.movie_factors(x[:, 1])
        return (users * movies).sum(dim=1)
    
    

In [10]:
x, y = dls.one_batch()
x.shape, y.shape

(torch.Size([64, 2]), torch.Size([64, 1]))

In [14]:
model = DotProduct(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())

learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,1.274338,1.344509,00:05
1,1.020230,1.133973,00:04
2,0.872318,1.008438,00:07
3,0.751804,0.927293,00:07
4,0.746245,0.899897,00:05


In [25]:
# sigmoid to 0~5
class DotProduct(Module):
    def __init__(self, n_users, n_movies, n_factors, y_range=(0,5.5)):
        super().__init__()
        self.user_factors = Embedding(n_users, n_factors)
        self.movie_factors = Embedding(n_movies, n_factors)
        self.y_range = y_range
        
    def forward(self, x):
        users = self.user_factors(x[:, 0])
        movies = self.movie_factors(x[:, 1])
        return sigmoid_range((users * movies).sum(dim=1), *self.y_range)
    
model = DotProduct(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())

learn.fit_one_cycle(5, 5e-3)

RuntimeError: Exception occured in `TrainEvalCallback` when calling event `before_fit`:
	CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [17]:
# some users tends to rate lower
# some movies are shit
class DotProduct(Module):
    def __init__(self, n_users, n_movies, n_factors, y_range=(0,5.5)):
        super().__init__()
        self.user_factors = Embedding(n_users, n_factors)
        self.user_bias = Embedding(n_users, 1)
        self.movie_factors = Embedding(n_movies, n_factors)
        self.movie_bias = Embedding(n_movies, 1)
        self.y_range = y_range
        
    def forward(self, x):
        users = self.user_factors(x[:, 0])
        movies = self.movie_factors(x[:, 1])
        res = (users * movies).sum(dim=1, keepdim=True)
        res += self.user_bias(x[:, 0]) + self.movie_bias(x[:, 1])
        return sigmoid_range(res, *self.y_range)
    
model = DotProduct(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())

learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,0.868731,0.964459,00:05
1,0.592885,0.921888,00:09
2,0.405443,0.951485,00:07
3,0.322742,0.962438,00:07
4,0.299368,0.963251,00:09


In [18]:
# weight decay
model = DotProduct(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())

learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.902549,0.985335,00:05
1,0.658567,0.907515,00:05
2,0.539261,0.895409,00:06
3,0.441184,0.878276,00:09
4,0.428110,0.873606,00:09


In [13]:
def create_params(size):
    return nn.Parameter(torch.zeros(*size).normal_(0, 0.01))

class DotProduct(Module):
    def __init__(self, n_users, n_movies, n_factors, y_range=(0,5.5)):
        super().__init__()
        self.user_factors = create_params([n_users, n_factors])
        self.user_bias = create_params([n_users])
        self.movie_factors = create_params([n_movies, n_factors])
        self.movie_bias = create_params([n_movies])
        self.y_range = y_range
        
    def forward(self, x):
        users = self.user_factors[x[:, 0]]
        movies = self.movie_factors[x[:, 1]]
        res = (users * movies).sum(dim=1)
        res += self.user_bias[x[:, 0]] + self.movie_bias[x[:, 1]]
        return sigmoid_range(res, *self.y_range)
    
model = DotProduct(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())

learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.889746,0.963973,00:06
1,0.663640,0.888544,00:06
2,0.521961,0.871154,00:06
3,0.456587,0.855045,00:05
4,0.439027,0.851144,00:06


In [32]:
movie_bias = learn.model.movie_bias.squeeze()
idxs = movie_bias.argsort(descending=True)[:5]
[dls.classes['title'][i] for i in idxs]

["Schindler's List (1993)",
 'Shawshank Redemption, The (1994)',
 'Titanic (1997)',
 'Star Wars (1977)',
 'Good Will Hunting (1997)']

In [33]:
learn = collab_learner(dls, n_factors=50, y_range=(0, 5.5))
learn.fit_one_cycle(5,5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.897453,0.942804,00:06
1,0.687668,0.896356,00:05
2,0.510918,0.868585,00:05
3,0.443651,0.856350,00:05
4,0.441061,0.852408,00:05


In [34]:
learn.model

EmbeddingDotBias(
  (u_weight): Embedding(944, 50)
  (i_weight): Embedding(1665, 50)
  (u_bias): Embedding(944, 1)
  (i_bias): Embedding(1665, 1)
)

In [36]:
movie_bias = learn.model.i_bias.weight.squeeze()
idxs = movie_bias.argsort(descending=True)[:5]
[dls.classes['title'][i] for i in idxs]

['Good Will Hunting (1997)',
 "Schindler's List (1993)",
 'Star Wars (1977)',
 'As Good As It Gets (1997)',
 'Titanic (1997)']

In [43]:
# distance
movie_factors = learn.model.i_weight.weight
idx = dls.classes['title'].o2i['Star Wars (1977)']
distances = nn.CosineSimilarity(dim=1)(movie_factors, movie_factors[idx][None])
idx = distances.argsort(descending=True)[1]
dls.classes['title'][idx]

'Return of the Jedi (1983)'

In [60]:
# deep learning
embs = get_emb_sz(dls)
embs

[(944, 74), (1665, 102)]

In [65]:
class CollabNN(Module):
    def __init__(self, user_sz, item_sz, y_range=(0, 5.5), n_act=100):
        super().__init__()
        self.user_factors = Embedding(*user_sz)
        self.item_factors = Embedding(*item_sz)
        self.layers = nn.Sequential(
            nn.Linear(user_sz[1] + item_sz[1], n_act),
            nn.ReLU(),
            nn.Linear(n_act, 1)
        )
        self.y_range = y_range
        
    def forward(self, x):
        embs = self.user_factors(x[:, 0]), self.item_factors(x[:, 1])
        x = self.layers(torch.cat(embs, dim=1))
        return sigmoid_range(x, *self.y_range)

In [66]:
model = CollabNN(*embs)

In [67]:
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3, wd=0.01)

epoch,train_loss,valid_loss,time
0,0.906004,0.965521,00:07
1,0.849285,0.910773,00:08
2,0.786637,0.885473,00:10
3,0.765756,0.875741,00:10
4,0.718167,0.873057,00:10


In [52]:
learn = collab_learner(dls, use_nn=True, y_range=(0,5.5), layers=[100, 50])
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.969644,0.978405,00:07
1,0.905084,0.920007,00:07
2,0.823467,0.888766,00:07
3,0.744446,0.866340,00:07
4,0.695954,0.866460,00:07
